# Upload the Model

To upload the Llama model to s3-compatible storage so that you can deploy it the the model serving server.

## Install the required packages and define a function for the upload

In [1]:
!pip install boto3 botocore


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
import boto3
import botocore

aws_access_key_id = 'minio'#os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = 'minio123'#os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = 'http://minio.ic-shared-minio.svc:9000'#os.environ.get('AWS_S3_ENDPOINT')
region_name = 'us-east-1'#os.environ.get('AWS_DEFAULT_REGION')
bucket_name = 'llama-2-weights'#os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files

def convert_model_id(model_id: str) -> str:
    return f"/models--{model_id.replace('/', '--')}"

def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

## Verify the upload

In your S3 bucket, under the `models` upload prefix, run the `list_object` command. 

The `meta-llama/Llama-3.2-1B` model will convert to `models--meta-llama--Llama-3.2-1B` and  upload to the `llama-2-weights` bucket. 

If this is the first time running the code, this cell will have no output.

If you've already uploaded your model, you should see this output: `models--meta-llama--Llama-3.2-1B/**`


In [3]:
list_objects('models')

## Upload and check again

Use the function to upload the `models` folder in a rescursive fashion:

In [9]:
local_models_directory = "/tmp/Llama-3.2-1B/"
model_id = "meta-llama/Llama-3.2-1B"
s3_prefix = convert_model_id(model_id)

if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist.")

num_files = upload_directory_to_s3(local_models_directory, s3_prefix)

if num_files == 0:
    raise ValueError("No files uploaded.")


/tmp/Llama-3.2-1B/.gitattributes -> /models--meta-llama--Llama-3.2-1B/.gitattributes
/tmp/Llama-3.2-1B/tokenizer_config.json -> /models--meta-llama--Llama-3.2-1B/tokenizer_config.json
/tmp/Llama-3.2-1B/tokenizer.json -> /models--meta-llama--Llama-3.2-1B/tokenizer.json
/tmp/Llama-3.2-1B/config.json -> /models--meta-llama--Llama-3.2-1B/config.json
/tmp/Llama-3.2-1B/USE_POLICY.md -> /models--meta-llama--Llama-3.2-1B/USE_POLICY.md
/tmp/Llama-3.2-1B/README.md -> /models--meta-llama--Llama-3.2-1B/README.md
/tmp/Llama-3.2-1B/model.safetensors -> /models--meta-llama--Llama-3.2-1B/model.safetensors
/tmp/Llama-3.2-1B/special_tokens_map.json -> /models--meta-llama--Llama-3.2-1B/special_tokens_map.json
/tmp/Llama-3.2-1B/generation_config.json -> /models--meta-llama--Llama-3.2-1B/generation_config.json
/tmp/Llama-3.2-1B/LICENSE.txt -> /models--meta-llama--Llama-3.2-1B/LICENSE.txt
/tmp/Llama-3.2-1B/original/params.json -> /models--meta-llama--Llama-3.2-1B/original/params.json
/tmp/Llama-3.2-1B/origi

To confirm this worked, run the `list_objects` function again:

In [5]:
list_objects("models")

models--meta-llama--Llama-3.2-1B/.cache/huggingface/.gitignore
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/.gitattributes.lock
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/.gitattributes.metadata
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/LICENSE.txt.lock
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/LICENSE.txt.metadata
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/README.md.lock
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/README.md.metadata
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/USE_POLICY.md.lock
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/USE_POLICY.md.metadata
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/config.json.lock
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/config.json.metadata
models--meta-llama--Llama-3.2-1B/.cache/huggingface/download/generation_config.json.lock
models--meta-llama--Llama-3.2-1B/.cache/hu

### Next Step

Now that you've saved the model to s3 storage, you can refer to the model by using the same data connection to serve the model as an API.
